In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers as esHelper
import json
import config
import os
import re
import shutil
esconfig = config.ES_CONFIG
DATA_FOLDER = '../data/es_data/'

In [19]:
import tarfile
tar = tarfile.open("../data/titles_04.archive.tar.gz")
tarFiles = tar.getmembers()
tar.extractall(path=DATA_FOLDER)
list(map(lambda file: ''.join([DATA_FOLDER, file]), os.listdir(DATA_FOLDER)))

['../data/es_data/titles.47.json',
 '../data/es_data/titles.45.json',
 '../data/es_data/titles.22.json',
 '../data/es_data/titles.23.json']

In [5]:
es = Elasticsearch( hosts= esconfig['host'])
es.info()
es.nodes.info()
#es.indices.delete(index='imdb', ignore=[400, 404])
#es.indices.create(index='imdb', ignore=400)
#es.indices.put_mapping(index='imdb', doc_type='titles', body=json.load(open('../data/imdb.title.mapping.json')))

{'acknowledged': True}

In [ ]:
#es.indices.get_mapping(index='imdb')

In [10]:
from helper.es_helper import ESHelper
esHelper = ESHelper(esconfig)

In [11]:
esHelper.info()

{'cluster_name': 'cluster-es',
 'cluster_uuid': 'AxFHMCPFQmGNBxrl8rPpXA',
 'name': 'node-es',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2018-02-16T19:01:30.685723Z',
  'build_hash': '10b1edd',
  'build_snapshot': False,
  'lucene_version': '7.2.1',
  'minimum_index_compatibility_version': '5.0.0',
  'minimum_wire_compatibility_version': '5.6.0',
  'number': '6.2.2'}}

In [20]:
titles = list(map(lambda file: ''.join([DATA_FOLDER, file]), os.listdir(DATA_FOLDER)))

def prep_bulk_data(data):
            for item in data:                
                yield {
                       '_index': 'imdb',
                       '_type': 'titles',
                       '_id': item['tconst'],
                       '_source': item
                }
for title in titles:
    try:        
        print('Working on file: {}'.format(title))
        esHelper.bulk_stream(prep_bulk_data(json.load(open(title))))    
        os.remove(title)
    except Exception as ex :
        print ('Ingnoring error {}'.format(ex));
               

Working on file: ../data/es_data/titles.47.json
Failed to record event: {'error': {'reason': 'failed to parse [runtimeMinutes]', 'caused_by': {'reason': 'For input string: "Animation,Family"', 'type': 'number_format_exception'}, 'type': 'mapper_parsing_exception'}, 'status': 400, '_index': 'imdb', '_type': 'titles', '_id': 'tt7841930'}
Success count: 99999, Failure count: 1
Working on file: ../data/es_data/titles.45.json
Failed to record event: {'error': {'reason': 'failed to parse [runtimeMinutes]', 'caused_by': {'reason': 'For input string: "Action,Adventure,Animation"', 'type': 'number_format_exception'}, 'type': 'mapper_parsing_exception'}, 'status': 400, '_index': 'imdb', '_type': 'titles', '_id': 'tt7406458'}
Success count: 99999, Failure count: 1
Working on file: ../data/es_data/titles.22.json
Success count: 100000, Failure count: 0
Working on file: ../data/es_data/titles.23.json
Failed to record event: {'error': {'reason': 'failed to parse [runtimeMinutes]', 'caused_by': {'reas

In [ ]:
import json
def prep_bulk_data(data):
            for item in data:                
                yield {
                       '_index': 'imdb',
                       '_type': 'titles',
                       '_id': item['tconst'],
                       '_source': item
                }

#esHelper.stream_bulk(prep_bulk_data(json.load(open('./data/titles.0.json'))))
x = prep_bulk_data(json.load(open('../data/es_data/titles.23.json')))